# Trait Simulation - VCM Power Example

Authors: Sarah Ji, Janet Sinsheimer, Kenneth Lange, Hua Zhou

In this notebook we show how to use the `TraitSimulation.jl` package we illustrate how TraitSimulation.jl can easily simulate traits from genotype data, all within the OpenMendel universe. Operating within this universe brings potential advantages over the available software(s) when needed for downstream analysis or study design. 

Using just a few calls on the command line to the appropriate packages within the OpenMendel, we demonstrate in three easy examples the utilities of the TraitSimulation.jl package.

Reference on Introduction to Variance Component Models in Genetic Setting, textbook link.


## Background

There is a lack of software available to geneticists who wish to calculate power and sample sizes in designing a study on genetics data. Typically, the study power depends on assumptions about the underlying disease model.  Many power calculating software tools operate as a black box and do not allow for customization.  To develop custom tests, researchers can develop their own simulation procedures to carry out power calculations.  One limitation with many existing methods for simulating traits conditional on genotypes is that these methods are limited to normally distributed traits and to fixed effects. 

This software package, TraitSimuliation.jl addresses the need for simulated trait data in genetic analyses.  This package generates data sets that will allow researchers to accurately check the validity of programs and to calculate power for their proposed studies. This package gives users the ability to easily simulate phenotypic traits under generalized linear models (GLMs) or variance component models (VCMs) conditional on PLINK formatted genotype data [3]. In addition, we include customized simulation utilities that accompany specific genetic analysis options in Open-Mendel; for example, ordered, multinomial traits. We demonstrate these simulation utilities on the example dataset described below.


## Demonstration

##### Example Data

We use the OpenMendel package [SnpArrays.jl](https://openmendel.github.io/SnpArrays.jl/latest/) to both read in and write out PLINK formatted files. 

Based on several different features in the EHR including diabetes diagnostic codes, diabetes medication, hyperglycemia in blood results defined by HbA1c and fasting glucose levels, and presence of diabetes process of care codes, the algorithm categorizes individuals into different categories that relate to how likely they are to have diabetes. 

For convenience we use the common assumption that the residual covariance among two relatives can be captured by the additive genetic variance times twice the kinship coefficient.

In each example the user can specify the simulation model parameters, along with the number of repitions for each simulation model as desired. By default, the simulation will return the result of a single simulation.


### Double check that you are using Julia version 1.0 or higher by checking the machine information

In [28]:
versioninfo()

Julia Version 1.3.0
Commit 46ce4d7933 (2019-11-26 06:09 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i9-9920X CPU @ 3.50GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)


In [29]:
using Random, Plots, DataFrames, LinearAlgebra, StatsFuns, CSV
using SnpArrays, TraitSimulation, GLM, StatsBase, VarianceComponentModels
Random.seed!(1234);

# Reading genotype data using SnpArrays

First use `SnpArrays.jl` to read in the genotype data. We use PLINK formatted data with the same prefixes for the .bim, .fam, .bed files.

SnpArrays is a very useful utility and can do a lot more than just read in the data. More information about all the functionality of SnpArrays can be found at:
https://openmendel.github.io/SnpArrays.jl/latest/

As missing genotypes are often due to problems making the calls, the called genotypes at a marker with too much missing genotypes are potentially unreliable. By default, SnpArrays filters to keep only the genotypes with success rates greater than 0.98 and the minimum minor allele frequency to be 0.01. If the user wishes to change the stringency, change the number given in filter according to [SnpArrays](https://openmendel.github.io/SnpArrays.jl/latest/#Fitering-1).

In [33]:
filename = "fullhypertensionbad"
full_snps = SnpArray("fullhypertensionbad.bed", 3754)

3754×470228 SnpArray:
 0x03  0x03  0x03  0x02  0x03  0x03  …  0x00  0x00  0x00  0x00  0x00  0x00
 0x02  0x03  0x02  0x03  0x03  0x02     0x00  0x00  0x00  0x00  0x00  0x00
 0x03  0x03  0x03  0x03  0x03  0x03     0x00  0x00  0x00  0x00  0x00  0x00
 0x02  0x02  0x03  0x03  0x03  0x03     0x00  0x00  0x00  0x00  0x00  0x00
 0x03  0x03  0x03  0x02  0x03  0x03     0x00  0x00  0x00  0x00  0x00  0x00
 0x02  0x03  0x02  0x03  0x03  0x02  …  0x00  0x00  0x00  0x00  0x00  0x00
 0x03  0x03  0x03  0x03  0x03  0x03     0x00  0x00  0x00  0x00  0x00  0x00
 0x02  0x03  0x02  0x03  0x03  0x02     0x00  0x00  0x00  0x00  0x00  0x00
 0x02  0x03  0x02  0x03  0x03  0x02     0x00  0x00  0x00  0x00  0x00  0x00
 0x03  0x03  0x03  0x03  0x03  0x03     0x00  0x00  0x00  0x00  0x00  0x00
 0x03  0x03  0x03  0x03  0x03  0x03  …  0x00  0x00  0x00  0x00  0x00  0x00
 0x03  0x03  0x03  0x03  0x03  0x03     0x00  0x00  0x00  0x00  0x00  0x00
 0x02  0x03  0x02  0x03  0x03  0x02     0x00  0x00  0x00  0x00  0x00  0x00
   

In [31]:
sample_snp_data = SnpData(filename)

SnpData(people: 3754, snps: 470228,
snp_info: 
│ Row │ chromosome │ snpid       │ genetic_distance │ position │ allele1      │ allele2      │
│     │ String     │ String      │ Float64          │ Int64    │ Categorical… │ Categorical… │
├─────┼────────────┼─────────────┼──────────────────┼──────────┼──────────────┼──────────────┤
│ 1   │ 1          │ rs3131972   │ 0.0              │ 752721   │ A            │ G            │
│ 2   │ 1          │ rs12184325  │ 0.0              │ 754105   │ T            │ C            │
│ 3   │ 1          │ rs3131962   │ 0.0              │ 756604   │ A            │ G            │
│ 4   │ 1          │ rs12562034  │ 0.0              │ 768448   │ A            │ G            │
│ 5   │ 1          │ rs116390263 │ 0.0              │ 772927   │ T            │ C            │
│ 6   │ 1          │ rs4040617   │ 0.0              │ 779322   │ G            │ A            │
…,
person_info: 
│ Row │ fid       │ iid       │ father    │ mother    │ sex       │ phenotype │
│

The published hypertension GWAS analysis includes the following covariates: sex, center, age, age2, BMI, and the top ten principal components to adjust for ancestry/relatedness.


# Construct Genetic Model: (VCM Parameter Specification)

Say that you have the the classical setting in genetics, two variance components, one for the additive genetic variance and one for the environmental variance. For simulating $d$ correlated traits for $n$ related people under the VCM, users can specify their covariance structures as follows.

$$Y_{n \times d} \sim \text{MatrixNormal}(\mathbf{M}_{n \times d} = XB, \Omega_{nd \times nd} = \Sigma_A \otimes V_A + \Sigma_E \otimes V_E)$$

In [34]:
bimfile = sample_snp_data.snp_info # store the snp_info with the snp names
snpid  = bimfile[!, :snpid] # store the snp names in the snpid vector
causal_snp_index = findall(x -> x == "rs11240779", snpid); # find the index of the snp of interest by snpid
causal_snp = @view full_snps[:, causal_snp_index]

3754×1 view(::SnpArray, :, [7]) with eltype UInt8:
 0x02
 0x02
 0x03
 0x02
 0x02
 0x02
 0x03
 0x02
 0x03
 0x03
 0x03
 0x03
 0x02
    ⋮
 0x03
 0x03
 0x02
 0x03
 0x03
 0x03
 0x03
 0x02
 0x03
 0x02
 0x02
 0x03

Here are the minor allele frequencies

In [40]:
maf_cs = maf(@view full_snps[:, causal_snp_index])[:]

1-element Array{Float64,1}:
 0.22996515679442509

In [42]:
# Generating Effect Sizes from Chisquared(df = 1) density
chisq_coeff = round.(chisqpdf.(1, maf_cs), digits = 3)

1-element Array{Float64,1}:
 0.742

In [44]:
GRM = grm(full_snps, minmaf = 0.05)

3754×3754 Array{Float64,2}:
  0.502981      0.000936836   0.000954635  …  -0.00105728    0.000601558
  0.000936836   0.49314      -0.00087355      -0.000417225  -0.00400204 
  0.000954635  -0.00087355    0.499788         0.00158897   -0.000694345
 -0.00276147    0.000671317  -0.00228694       0.000548168   0.0027718  
  0.0036049    -0.000643086  -0.00354798       0.00240853   -0.00151607 
 -0.00378644    0.00418099   -0.00077422   …  -0.000466577  -0.000401483
 -0.00432779   -0.00593195    0.00209739       0.00366185    0.00421288 
  0.00043021   -0.000119469  -8.9781e-5       -0.00218583   -0.00199718 
 -4.86011e-5   -0.00306901   -0.00267965      -0.000654534   0.00192883 
  0.00242187    0.00625245   -0.00419928      -0.00108094   -0.00293948 
  0.00104979    0.00316535    0.0013463    …   0.00493102   -0.00126338 
  0.00236206   -0.000116416   0.00203903      -0.00178021   -0.00293362 
  0.0041086    -0.00485801    0.00125295       0.000377517   0.000547523
  ⋮                    

# Power Calculation

Now we show how to simulate from customized simulation models that accompany specific genetic analysis options in OpenMendel; for example, ordered, multinomial traits and Variance Component Models.


This example illustrates the use of the simulations to generates data sets allowing researchers to accurately check the validity of programs and to calculate power for their proposed studies. 

We illustrate this example in three digestable steps: 
   * The first by simulating genotypes and covariate values representative for our study population.
   * Carry over the simulated design matrix from (1) to create the OrderedMultinomialTrait model object.
   * Simulate off the OrderedMultinomialTrait model object created in (2) and run the power analyses for the desired significance level.


### convert
By default `convert` function translates genotypes according to the *additive* SNP model, which essentially counts the number of **A2** allele (0, 1 or 2) per genotype. Other SNP models are *dominant* and *recessive*, both in terms of the **A2** allele.

| Genotype | `SnpArray` | `model=ADDITIVE_MODEL` | `model=DOMINANT_MODEL` | `model=RECESSIVE_MODEL` |    
|:---:|:---:|:---:|:---:|:---:|  
| A1,A1 | 0x00 | 0 | 0 | 0 |  
| missing | 0x01 | NaN | NaN | NaN |
| A1,A2 | 0x02 | 1 | 1 | 0 |  
| A2,A2 | 0x03 | 2 | 1 | 1 |  

If desired, the user can decide to specify alternative model parameters found in the [SnpArrays](https://openmendel.github.io/SnpArrays.jl/latest/#convert-and-copyto!-1) documentation.

In [57]:
locus = convert(Vector{Float64}, causal_snp[:])

3754-element Array{Float64,1}:
 2.0
 2.0
 3.0
 2.0
 2.0
 2.0
 3.0
 2.0
 3.0
 3.0
 3.0
 3.0
 2.0
 ⋮  
 3.0
 3.0
 2.0
 3.0
 3.0
 3.0
 3.0
 2.0
 3.0
 2.0
 2.0
 3.0

## Design Matrix

Now we concatenate the simulated causal locus, transformed to Float64, to the other covariates we want to control for in the study design.

The published hypertension GWAS analysis includes the following covariates: sex, center, age, age2, BMI, and the top ten principal components to adjust for ancestry/relatedness.

In [58]:
n = length(locus)
published_covariate_data = CSV.read("/mnt/UKBiobank/ukbdata/ordinalanalysis/Covariate_Final.csv")
covariates = published_covariate_data[1:n, 1:10]

,FID,IID,sex,center,age,age2,chip,bmi,hyptens,AveSBP
,Int64,Int64,Int64,Int64,Int64,Int64,String,Float64,Int64,Float64
1,1000019,1000019,1,11016,62,3844,ChipOne,29.3424,0,118.0
2,1000078,1000078,1,11009,51,2601,ChipOne,31.0375,2,134.0
3,1000081,1000081,1,11014,47,2209,ChipOne,23.335,2,132.5
4,1000105,1000105,0,11010,42,1764,ChipOne,21.5793,0,104.5
5,1000112,1000112,1,11021,55,3025,ChipOne,26.7327,2,139.0
6,1000129,1000129,1,11009,63,3969,ChipOne,26.7034,3,141.0
7,1000141,1000141,0,11008,51,2601,ChipOne,35.1706,1,128.0
8,1000164,1000164,0,11011,45,2025,ChipOne,25.235,1,122.0
9,1000224,1000224,1,11010,52,2704,ChipOne,31.6387,3,158.0


In [59]:
age = Float64.(covariates[!, :age])
sex = Float64.(covariates[!, :sex])
center = Int64.(covariates[!, :center])
bmi = Float64.(covariates[!, :bmi])
avg_SBP = Float64.(covariates[!, :AveSBP])

X_cov = DataFrame(sex = sex, center = center, age = age, age2 = age.^2, bmi = bmi)
Data = DataFrame(y = avg_SBP, sex = sex, center = center, age = age, age2 = age.^2, bmi = bmi, locus = locus)
X_full = hcat(X_cov, DataFrame(locus = locus))

,sex,center,age,age2,bmi,locus
,Float64,Int64,Float64,Float64,Float64,Float64
1,1.0,11016,62.0,3844.0,29.3424,2.0
2,1.0,11009,51.0,2601.0,31.0375,2.0
3,1.0,11014,47.0,2209.0,23.335,3.0
4,0.0,11010,42.0,1764.0,21.5793,2.0
5,1.0,11021,55.0,3025.0,26.7327,2.0
6,1.0,11009,63.0,3969.0,26.7034,2.0
7,0.0,11008,51.0,2601.0,35.1706,3.0
8,0.0,11011,45.0,2025.0,25.235,2.0
9,1.0,11010,52.0,2704.0,31.6387,3.0


## Phenotype Simulation:

Now that we have our simulated design matrix with the desired SNP of interest, we can simulate our phenotypes on the simulated data under different TraitSimulation models. To illustrate, we demonstrate on the `VCMTrait` model object in TraitSimulation.jl.

In [64]:
β_init = 
[4.435
  0.006
 -0.081
  0.007
  0.754]

5-element Array{Float64,1}:
  4.435
  0.006
 -0.081
  0.007
  0.754

In [65]:
n = size(GRM, 1)
I_n = Matrix{Float64}(I, n, n)

totalvc = @vc [0.1][:, :] ⊗ GRM + [0.9][:, :] ⊗ I_n
genetic_model = VCMTrait(Matrix(X_cov), β_init[:,:], totalvc)

Variance Component Model
  * number of traits: 1
  * number of variance components: 2
  * sample size: 3754

In [66]:
β_full = vcat(β_init, chisq_coeff)

6-element Array{Float64,1}:
  4.435
  0.006
 -0.081
  0.007
  0.754
  0.742

In [67]:
genetic_model_test = VCMTrait(Matrix(X_full), β_full[:, :], totalvc)

Variance Component Model
  * number of traits: 1
  * number of variance components: 2
  * sample size: 3754

## `Power Calculation:`

We use the following function to generate the p-values for the simulated power example for the ordered multinomial regression model. We range effect sizes in the vector γs, which collects effect sizes from 0 to 1 in increments of 0.05. As expected, the power increases as the effect size increases. 
    

In [68]:
γs = collect(0.0:0.05:0.5)

11-element Array{Float64,1}:
 0.0 
 0.05
 0.1 
 0.15
 0.2 
 0.25
 0.3 
 0.35
 0.4 
 0.45
 0.5 

Suppose we want to see a particular SNP has an effect on a given phenotype after accounting for relatedness among individuals. Here we fit variance component model with a single SNP *s* as fixed effect. 

$$\hspace{5em}  \mathbf{y} = \mathbf{X}\mathbf{\beta} + \mathbf{G}_s \gamma + \mathbf{g} + \mathbf{\epsilon} \hspace{5em} (1)$$

\begin{equation}
\begin{array}{ll}
\mathbf{g} \sim N(\mathbf{0}, \sigma_g^2\mathbf{\Phi}) \\
\mathbf{\epsilon} \sim N(\mathbf{0}, \sigma_e^2\mathbf{I})
\end{array}
\end{equation}

where 

* $\mathbf{y}$: phenotype 



and 


* Fixed effects:
    * $\mathbf{X}$: matrix of covariates including intercept
    * $\beta$: vector of covariate effects, including intercept
    * $\mathbf{G}_s$: genotype of SNP *s*
    * $\gamma$: (scalar) association parameter of interest, measuring the effect of genotype on phenotype  
* Random effects:
    * $\mathbf{g}$: random vector of polygenic effects with $\mathbf{g} \sim N(\mathbf{0}, \sigma_g^2 \mathbf{\Phi})$
        * $\sigma_g^2$: additive genetic variance
        * $\mathbf{\Phi}$: matrix of pairwise measures of genetic relatedness 
    * $\epsilon$: random vector with $\epsilon \sim N(\mathbf{0}, \sigma_e^2\mathbf{I})$
        * $\sigma_e^2$: non-genetic variance due to non-genetic effects assumed to be acting independently on individuals



To test whether SNP *s* is associated with phenotype, we fit two models. First consider the model without SNP *s* as fixed effects (aka null model): 

$$\hspace{5em}  \mathbf{y} = \mathbf{X}\mathbf{\beta} + \mathbf{g} + \mathbf{\epsilon} \hspace{5em} (2)$$

and the model with SNP *s* as fixed effects (1). Then we can compare the log likelihood to see if there is improvement in the model fit with inclusion of the SNP of interest. 

Each column of this matrix represents each of the detected effect sizes, and each row of this matrix represents each simulation for that effect size. The user feeds into the function the number of simulations, the vector of effect sizes, the TraitSimulation.jl model object, and the random seed.

For GLMTrait objects, the `realistic_power_simulation` function makes the appropriate calls to the GLM.jl package to get the simulation p-values obtained from testing the significance of the causal locus using the Wald Test by default. However since the GLM.jl package has its limitations, we include additional power utilities that make the appropriate function calls to the [OrdinalMultinomialModels](https://openmendel.github.io/OrdinalMultinomialModels.jl/stable/#Syntax-1) to get the p-value obtained from testing the significance of the causal locus.


For each effect size in $\gamma_s,$ in each column we have the p-values obtained from testing the significance of the causal locus `nsim = 100` times under the VCM, and the `randomseed = 1234`.

In [ ]:
nsim = 1000
randomseed = 1234
simulated_pvalues = power_simulation(nsim, γs, genetic_model_test, β_full[:, :], randomseed);
rename!(DataFrame(simulated_pvalues), [Symbol("γs = $(γs[i])") for i in 1:length(γs)])


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



,γs = 0.0,γs = 0.05,γs = 0.1,γs = 0.15,γs = 0.2,γs = 0.25,γs = 0.3
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.524195,0.000332155,0.000121027,4.61311e-15,5.4629e-23,2.85855e-47,2.73095e-42
2,0.998479,0.000355733,2.88285e-7,1.8455e-24,6.2421e-22,8.35046e-42,2.27117e-49
3,0.164856,0.182795,3.97841e-5,8.52498e-10,1.19908e-19,4.82556e-39,3.97908e-48
4,0.836477,0.0292655,3.36015e-5,2.37606e-11,1.43864e-17,4.62361e-40,1.43978e-39
5,0.614246,0.0274985,1.94474e-8,1.75697e-19,2.73971e-24,4.85311e-32,7.53796e-43
6,0.287576,0.0274272,3.02843e-5,7.1158e-18,3.52477e-25,2.80054e-31,1.83542e-49
7,0.484015,0.0220593,9.56956e-5,2.27652e-13,2.85198e-24,4.75938e-32,1.41651e-47
8,0.0267504,0.00680909,2.5878e-6,1.2997e-11,1.96043e-19,1.03943e-30,1.28455e-29
9,0.179694,0.124784,0.000282669,2.99293e-16,5.30596e-20,4.64257e-37,1.83945e-46


Now we find the power of each effect size in the user-specified γs vector at the specified alpha level of significance, and plot the trajectory using the Plots.jl package.

In [ ]:
α = 0.0000000005
power_effectsize = power(simulated_pvalues, α)

In [ ]:
plot(γs, power_effectsize, title = "VCM Power", label = "maf = $maf_cs, alpha = $α", lw = 3 , legend = :bottomright, legendfontsize= 9)  # plot power
xlabel!("Detectable Effect Size")
hline!([.8], label = "power = 80%", lw = 3)
vline!([.29], label = "minimum detectable effect size = 0.29")
savefig("/home/sarahji/TraitSimulation.jl/docs/vcmpower.pdf")

## Citations: 

[1] Lange K, Papp JC, Sinsheimer JS, Sripracha R, Zhou H, Sobel EM (2013) Mendel: The Swiss army knife of genetic analysis programs. Bioinformatics 29:1568-1570.`


[2] OPENMENDEL: a cooperative programming project for statistical genetics.
[Hum Genet. 2019 Mar 26. doi: 10.1007/s00439-019-02001-z](https://www.ncbi.nlm.nih.gov/pubmed/?term=OPENMENDEL).

[3] German, CA, Sinsheimer, JS, Klimentidis, YC, Zhou, H, Zhou, JJ. Ordered multinomial regression for genetic association analysis of ordinal phenotypes at Biobank scale. Genetic Epidemiology. 2019; 1– 13. https://doi.org/10.1002/gepi.22276
